In [ ]:
import sys
sys.path.append('/home/jiajunb/neural-dimension-reduction')

In [ ]:
import os

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from torch.nn import functional as F
from collections import OrderedDict

torch.manual_seed(0)

In [ ]:
train_data = torch.from_numpy(pd.read_csv('../data/processed/sample/train.csv', header=None).to_numpy()).double()
# train_loader = DataLoader(train_data, batch_size=1000, pin_memory=True)


In [ ]:
model = nn.Sequential(
        OrderedDict([
            ('fc0', nn.Linear(200, 20)),
#             ('bn0', nn.BatchNorm1d(200)),
#             ('relu0', nn.ReLU(inplace=True)),
#             ('fc0', nn.Linear(200, 500)),
#             ('bn1', nn.BatchNorm1d(500)),
#             ('relu1', nn.ReLU(inplace=True)),
#             ('fc1', nn.Linear(500, 100)),
#             ('bn2', nn.BatchNorm1d(100)),
#             ('relu2', nn.ReLU(inplace=True)),
#             ('fc2', nn.Linear(100, 20)),
#             ('bn3', nn.BatchNorm1d(20)),
#             ('relu3', nn.ReLU(inplace=True)),
#             ('fc3', nn.Linear(20, 20)),
#             ('bn4', nn.BatchNorm1d(20)),
#             ('relu4', nn.ReLU(inplace=True)),
#             ('fc4', nn.Linear(20, 20)),
#             ('bn5', nn.BatchNorm1d(20)),
#             ('relu5', nn.ReLU(inplace=True)),
#             ('fc5', nn.Linear(20, 20)),
        ])
)

In [ ]:
weight_decay = 1e-5
learning_rate = 1e-5
num_epoches = 40

In [ ]:
device = torch.device('cuda:0')

no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(
        nd in n for nd in no_decay) and p.requires_grad], 'weight_decay': weight_decay},
    {'params': [p for n, p in model.named_parameters() if any(
        nd in n for nd in no_decay) and p.requires_grad], 'weight_decay': 0.0}
]

optimizer = torch.optim.AdamW(params=optimizer_grouped_parameters, lr=learning_rate)
model = model.to(device)

In [ ]:
x_dist = torch.cdist(x1=train_data, x2=train_data, p=2).to(device)

In [ ]:
model.train()

for epoch_idx in range(num_epoches):
    embedded_x = model(train_data.to(torch.float).to(device))
    embedded_x = F.softmax(embedded_x, dim=1)
    embedded_x_dist = torch.cdist(x1=embedded_x, x2=embedded_x, p=2)
    loss = torch.sum(x_dist * torch.log(x_dist / (embedded_x_dist + 1e-9)))
    model.zero_grad()
    loss.backward()
    optimizer.step()
    print(embedded_x.max(dim=1))
    print(embedded_x_dist)
    print(f'[{epoch_idx}]/[{num_epoches}] {loss.item()}')